<a href="https://colab.research.google.com/github/BorisLoveDev/don-t_know-s_agents/blob/main/solo_research_best_aug_cnn_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Импорт необходимых библиотек
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import numpy as np
from google.colab import drive
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle


In [2]:
# Монтирование Google Drive
drive.mount('/content/drive')

# Путь к папке для загрузки данных
data_path = '/content/drive/My Drive/solo_research_aug_cnn/datasets'

Mounted at /content/drive


In [14]:
# Загрузка сохраненных данных
x_train_new = np.load(os.path.join(data_path, 'x_train_new.npy'))
y_train_new = np.load(os.path.join(data_path, 'y_train_new.npy'))
x_val = np.load(os.path.join(data_path, 'x_val.npy'))
y_val = np.load(os.path.join(data_path, 'y_val.npy'))
x_test = np.load(os.path.join(data_path, 'x_test.npy'))
y_test = np.load(os.path.join(data_path, 'y_test.npy'))

# Загрузка дополнительных наборов данных
x_val_easy = np.load(os.path.join(data_path, 'x_val_easy.npy'))
x_val_medium = np.load(os.path.join(data_path, 'x_val_medium.npy'))
x_val_hard = np.load(os.path.join(data_path, 'x_val_hard.npy'))
x_val_indistinguishable = np.load(os.path.join(data_path, 'x_val_indistinguishable.npy'))


# Перемешивание данных
x_val, y_val = shuffle(x_val, y_val, random_state=42)
# Остальные данные уже перемешенные

# Применение one-hot encoding к меткам
y_train_new_ohe = to_categorical(y_train_new, 10)
y_val_ohe = to_categorical(y_val, 10)
y_test_ohe = to_categorical(y_test, 10)

print("Data loaded and one-hot encoded successfully.")

Data loaded and one-hot encoded successfully.


In [4]:
# Печать форм всех загруженных и обработанных данных
print("Форма x_train_new:", x_train_new.shape)
print("Форма y_train_new:", y_train_new.shape)
print("Форма y_train_new_ohe:", y_train_new_ohe.shape)
print("Форма x_val:", x_val.shape)
print("Форма y_val:", y_val.shape)
print("Форма y_val_ohe:", y_val_ohe.shape)
print("Форма x_test:", x_test.shape)
print("Форма y_test:", y_test.shape)
print("Форма y_test_ohe:", y_test_ohe.shape)

# Печать форм дополнительных наборов данных
print("Форма x_val_easy:", x_val_easy.shape)
print("Форма x_val_medium:", x_val_medium.shape)
print("Форма x_val_hard:", x_val_hard.shape)
print("Форма x_val_indistinguishable:", x_val_indistinguishable.shape)

Форма x_train_new: (50000, 28, 28, 1)
Форма y_train_new: (50000,)
Форма y_train_new_ohe: (50000, 10)
Форма x_val: (10000, 28, 28, 1)
Форма y_val: (10000,)
Форма y_val_ohe: (10000, 10)
Форма x_test: (10000, 28, 28, 1)
Форма y_test: (10000,)
Форма y_test_ohe: (10000, 10)
Форма x_val_easy: (10000, 28, 28, 1)
Форма x_val_medium: (10000, 28, 28, 1)
Форма x_val_hard: (10000, 28, 28, 1)
Форма x_val_indistinguishable: (10000, 28, 28, 1)


In [5]:
# Путь к папке для сохранения модели
model_path = '/content/drive/My Drive/solo_research_aug_cnn/models'

# Путь к файлу модели
model_file = os.path.join(model_path, 'mnist_model.keras')
# Загрузка модели
model = tf.keras.models.load_model(model_file)

In [7]:
#x_val = np.load(os.path.join(data_path, 'x_val_идеальные.npy'))

In [20]:
# Список наборов данных различных уровней трудности
difficulty_datasets = {
    "неизменные": x_val,
    "легкие": x_val_easy,
    "средние": x_val_medium,
    "трудные": x_val_hard,
    "неразличимые": x_val_indistinguishable
}

# Оценка точности модели на валидационных данных для каждого уровня трудности
for difficulty, data in difficulty_datasets.items():
    test_loss, test_accuracy = model.evaluate(data, y_val_ohe, verbose=2)
    print(f"Точность модели на '{difficulty}' данных: {test_accuracy * 100:.2f}%")


313/313 - 1s - loss: 0.0159 - accuracy: 0.9946 - 854ms/epoch - 3ms/step
Точность модели на 'неизменные' данных: 99.46%
313/313 - 1s - loss: 0.0340 - accuracy: 0.9894 - 1s/epoch - 4ms/step
Точность модели на 'легкие' данных: 98.94%
313/313 - 1s - loss: 0.2048 - accuracy: 0.9335 - 1s/epoch - 4ms/step
Точность модели на 'средние' данных: 93.35%
313/313 - 1s - loss: 2.8698 - accuracy: 0.4139 - 1s/epoch - 4ms/step
Точность модели на 'трудные' данных: 41.39%
313/313 - 1s - loss: 5.9878 - accuracy: 0.1361 - 846ms/epoch - 3ms/step
Точность модели на 'неразличимые' данных: 13.61%


In [7]:
# Функция для деформации изображения
def deform_image_optimized(image, A, B, M=100, NP=5):
    C = M / (NP + 1.0)
    SS = np.zeros((M, M))
    XN, YN = np.zeros(M), np.zeros(M)
    DX, DY = np.linspace(0, M-1, M), np.linspace(0, M-1, M)

    for j in range(NP):
        TXN = (j + 0.5 - np.random.random()) * C
        TYN = (j + 0.5 - np.random.random()) * C
        TDX = (j + 0.5 - np.random.random()) * C
        TDY = (j + 0.5 - np.random.random()) * C
        AXN = B * (1.0 - 2.0 * np.random.random())
        AYN = B * (1.0 - 2.0 * np.random.random())
        ADX = A * (1.0 - 2.0 * np.random.random())
        ADY = A * (1.0 - 2.0 * np.random.random())
        PXN = (0.1 + 0.9 * np.random.random()) * C
        PYN = (0.1 + 0.9 * np.random.random()) * C
        PDX = (0.1 + 0.9 * np.random.random()) * C
        PDY = (0.1 + 0.9 * np.random.random()) * C

        DX += ADX * np.exp(-((DX - TDX) / PDX)**2)
        DY += ADY * np.exp(-((DY - TDY) / PDY)**2)
        XN += AXN * np.exp(-((DX - TXN) / PXN)**2)
        YN += AYN * np.exp(-((DY - TYN) / PYN)**2)

    # Применение деформации к изображению
    deformed_image = np.zeros((M, M))
    for j in range(M):
        for i in range(M):
            x_index = int(DX[i] + XN[j])
            y_index = int(DY[j] + YN[i])
            if 0 <= x_index < M and 0 <= y_index < M:
                deformed_image[j, i] = image[y_index, x_index] if y_index < image.shape[0] and x_index < image.shape[1] else 0

    return deformed_image

# Параметры деформации для разных уровней сложности
deform_params = {
    "идеальные": (0, 0),
    "легкие": (0.2, 1.0),
    "средние": (0.4, 2.0),
    "трудные": (0.8, 6.0),
    "неразличимые": (10.0, 30.0)
}

In [16]:
class DeformedImageDataGenerator:
    def __init__(self, x_data, y_data, batch_size, deformation_params):
        self.x_data = x_data
        self.y_data = y_data
        self.batch_size = batch_size
        self.deformation_params = deformation_params
        self.indexes = np.arange(len(self.x_data))

    def __len__(self):
        return int(np.floor(len(self.x_data) / self.batch_size))

    def __getitem__(self, index):
            batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
            x_batch = np.array([deform_image_optimized(self.x_data[i], *self.deformation_params) for i in batch_indexes])
            x_batch = np.expand_dims(x_batch, axis=-1)  # Добавление измерения канала
            x_batch = tf.image.resize(x_batch, (28, 28))  # Изменение размера изображений
            y_batch = self.y_data[batch_indexes]
            return x_batch, y_batch

    def on_epoch_end(self):
        np.random.shuffle(self.indexes)


In [19]:
# Параметры деформации для "средних" изображений
deformation_params_medium = deform_params["средние"]

# Создание генератора данных
batch_size = 1000  # можно изменить в зависимости от доступных ресурсов
data_generator = DeformedImageDataGenerator(x_val, y_val_ohe, batch_size, deformation_params_medium)

In [20]:
# Оценка точности на данных средней сложности
total_batches = len(data_generator)
total_accuracy = 0

for batch_index in range(total_batches):
    x_batch, y_batch = data_generator[batch_index]
    print(x_batch.shape)
    loss, accuracy = model.evaluate(x_batch, y_batch, verbose=0)
    total_accuracy += accuracy

average_accuracy = total_accuracy / total_batches
print(f"Средняя точность модели на данных средней сложности: {average_accuracy * 100:.2f}%")

(1000, 28, 28, 1)
(1000, 28, 28, 1)
(1000, 28, 28, 1)
(1000, 28, 28, 1)
(1000, 28, 28, 1)
(1000, 28, 28, 1)
(1000, 28, 28, 1)
(1000, 28, 28, 1)
(1000, 28, 28, 1)
(1000, 28, 28, 1)
Средняя точность модели на данных средней сложности: 10.00%
